In [1]:
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

#to split the data into test and train
from sklearn.model_selection import train_test_split

from random import randint

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

import random

from sklearn.metrics import accuracy_score

import pickle


In [2]:
bike_data=pd.read_csv("updated_bikedata.csv")
weather_data=pd.read_csv("updated_weatherdata.csv")

In [3]:
bike_data.head(3)

,Unnamed: 0,number,contract_name,name,bike_stands,available_bike_stands,available_bikes,status,last_update
0,0,42,dublin,SMITHFIELD NORTH,30,24,6,OPEN,2020-03-04 09:38:19
1,1,30,dublin,PARNELL SQUARE NORTH,20,19,1,OPEN,2020-03-04 09:30:14
2,2,54,dublin,CLONMEL STREET,33,14,19,OPEN,2020-03-04 09:39:15


In [4]:
weather_data.head(3)

,Unnamed: 0,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,weather_base,main_temp,feels_like,...,clouds_all,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,city_id,city_name,cod
0,0,-6.25,53.36,803,Clouds,broken clouds,04d,stations,274.81,271.01,...,75,2020-03-04 08:31:30,1,1565,IE,2020-03-04 07:05:02,2020-03-04 18:08:34,7778677,Dublin City,200
1,1,-6.25,53.36,803,Clouds,broken clouds,04d,stations,274.96,271.18,...,75,2020-03-04 08:45:02,1,1565,IE,2020-03-04 07:05:02,2020-03-04 18:08:34,7778677,Dublin City,200
2,3,-6.25,53.36,803,Clouds,broken clouds,04d,stations,275.38,272.64,...,75,2020-03-04 09:10:02,1,1565,IE,2020-03-04 07:05:02,2020-03-04 18:08:34,7778677,Dublin City,200


In [5]:
weather_data.shape

(3337, 26)

In [6]:
#convert the dateTime 
def date(x):
    formats = pd.to_datetime(x, errors='coerce')
    return formats

weather_data['dt'] = weather_data['dt'].apply(date)
weather_data['sys_sunrise'] = weather_data['sys_sunrise'].apply(date)
weather_data['sys_sunset'] = weather_data['sys_sunset'].apply(date)
bike_data['last_update'] = bike_data['last_update'].apply(date)

In [7]:
weather_data.dtypes

Unnamed: 0                      int64
coord_lon                     float64
coord_lat                     float64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
weather_base                   object
main_temp                     float64
feels_like                    float64
main_temp_min                 float64
main_temp_max                 float64
main_pressure                   int64
main_humidity                   int64
main_visibility                 int64
wind_speed                    float64
clouds_all                      int64
dt                     datetime64[ns]
sys_type                        int64
sys_id                          int64
sys_country                    object
sys_sunrise            datetime64[ns]
sys_sunset             datetime64[ns]
city_id                         int64
city_name                      object
cod                             int64
dtype: objec

In [8]:
# Select columns containing categorical data
weather_categorical_columns = weather_data[['weather_description', 'weather_icon',]].columns

# Convert data type to category for these columns
for column in weather_categorical_columns:
    weather_data[column] = weather_data[column].astype('category')

In [9]:
# weather_data = weather_data.drop('Unnamed: 0', 1)
# bike_data = bike_data.drop('Unnamed: 0', 1)

## Merging the two data frame

In [10]:
weather= weather_data[['weather_main','main_temp','main_pressure', 'main_humidity', 'wind_speed', 'dt']]

In [11]:
weather['dt'] = weather['dt'].apply(date)

C:\Users\olawa\Anaconda3\envs\comp47350py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
#sorting the data 
bike_data.sort_values('last_update', inplace=True)
weather.sort_values('dt', inplace=True)

C:\Users\olawa\Anaconda3\envs\comp47350py37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
weather.shape

(3337, 6)

In [14]:
weather.dtypes

weather_main             object
main_temp               float64
main_pressure             int64
main_humidity             int64
wind_speed              float64
dt               datetime64[ns]
dtype: object

In [15]:
weather.head()

,weather_main,main_temp,main_pressure,main_humidity,wind_speed,dt
0,Clouds,274.81,1006,68,1.92,2020-03-04 08:31:30
1,Clouds,274.96,1006,68,1.92,2020-03-04 08:45:02
2,Clouds,275.38,1006,68,0.50,2020-03-04 09:10:02
3,Clouds,276.20,1006,93,1.92,2020-03-04 09:25:28
4,Clouds,276.58,1006,93,1.92,2020-03-04 09:40:04


In [16]:
bike_data.head()

,Unnamed: 0,number,contract_name,name,bike_stands,available_bike_stands,available_bikes,status,last_update
69,69,95,dublin,ROYAL HOSPITAL,40,40,0,OPEN,2020-03-04 09:29:55
1,1,30,dublin,PARNELL SQUARE NORTH,20,19,1,OPEN,2020-03-04 09:30:14
40,40,77,dublin,WOLFE TONE STREET,29,29,0,OPEN,2020-03-04 09:30:29
77,77,12,dublin,ECCLES STREET,20,20,0,OPEN,2020-03-04 09:30:43
55,55,117,dublin,HANOVER QUAY EAST,40,14,26,OPEN,2020-03-04 09:30:45


In [17]:
#merging the data 
merged_data = pd.merge_asof(bike_data, weather, left_on='last_update', right_on='dt')

In [18]:
merged_data.head()

,Unnamed: 0,number,contract_name,name,bike_stands,available_bike_stands,available_bikes,status,last_update,weather_main,main_temp,main_pressure,main_humidity,wind_speed,dt
0,69,95,dublin,ROYAL HOSPITAL,40,40,0,OPEN,2020-03-04 09:29:55,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
1,1,30,dublin,PARNELL SQUARE NORTH,20,19,1,OPEN,2020-03-04 09:30:14,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
2,40,77,dublin,WOLFE TONE STREET,29,29,0,OPEN,2020-03-04 09:30:29,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
3,77,12,dublin,ECCLES STREET,20,20,0,OPEN,2020-03-04 09:30:43,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
4,55,117,dublin,HANOVER QUAY EAST,40,14,26,OPEN,2020-03-04 09:30:45,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28


In [19]:
# merged_data['Dates'] = pd.to_datetime(merged_data['last_update']).dt.date
# merged_data['Time'] = pd.to_datetime(merged_data['last_update']).dt.time

In [20]:
data=merged_data.drop(['Unnamed: 0', 'contract_name', 'dt','name','status'], axis=1)

In [21]:
#Save df to new csv file
data.to_csv('merged.csv')

## Training the data 

In [22]:
data.head()

,number,bike_stands,available_bike_stands,available_bikes,last_update,weather_main,main_temp,main_pressure,main_humidity,wind_speed
0,95,40,40,0,2020-03-04 09:29:55,Clouds,276.2,1006,93,1.92
1,30,20,19,1,2020-03-04 09:30:14,Clouds,276.2,1006,93,1.92
2,77,29,29,0,2020-03-04 09:30:29,Clouds,276.2,1006,93,1.92
3,12,20,20,0,2020-03-04 09:30:43,Clouds,276.2,1006,93,1.92
4,117,40,14,26,2020-03-04 09:30:45,Clouds,276.2,1006,93,1.92


In [23]:
data['year'] = data['last_update'].dt.year
data['month'] = data['last_update'].dt.month
data['day'] = data['last_update'].dt.day
data['hour'] = data['last_update'].dt.hour
data['minute'] = data['last_update'].dt.minute
data['dayofweek'] = data['last_update'].dt.dayofweek

In [24]:
data.head()

,number,bike_stands,available_bike_stands,available_bikes,last_update,weather_main,main_temp,main_pressure,main_humidity,wind_speed,year,month,day,hour,minute,dayofweek
0,95,40,40,0,2020-03-04 09:29:55,Clouds,276.2,1006,93,1.92,2020,3,4,9,29,2
1,30,20,19,1,2020-03-04 09:30:14,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2
2,77,29,29,0,2020-03-04 09:30:29,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2
3,12,20,20,0,2020-03-04 09:30:43,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2
4,117,40,14,26,2020-03-04 09:30:45,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2


In [25]:
data = data.drop('weather_main', 1)
data = data.drop('last_update', 1)

In [26]:
#input data 
input_data=data.drop(['available_bikes', 'available_bike_stands','bike_stands',], axis=1)

#target variable
target=data['available_bikes']


In [27]:
# Create 7 dataframes based on day of week
monday = data[data.dayofweek == 0]
tuesday = data[data.dayofweek == 1]
wednesday = data[data.dayofweek == 2]
thursday = data[data.dayofweek == 3]
friday = data[data.dayofweek == 4]
saturday = data[data.dayofweek == 5]
sunday = data[data.dayofweek == 6]

In [28]:
monday

,number,bike_stands,available_bike_stands,available_bikes,main_temp,main_pressure,main_humidity,wind_speed,year,month,day,hour,minute,dayofweek
84546,67,40,5,35,278.28,1004,80,6.7,2020,3,9,0,0,0
84547,61,25,12,13,278.28,1004,80,6.7,2020,3,9,0,0,0
84548,45,30,7,23,278.28,1004,80,6.7,2020,3,9,0,0,0
84549,47,40,32,8,278.28,1004,80,6.7,2020,3,9,0,0,0
84550,106,40,40,0,278.28,1004,80,6.7,2020,3,9,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228585,78,40,20,20,282.11,1015,81,7.2,2020,3,16,19,33,0
228586,102,40,23,17,282.11,1015,81,7.2,2020,3,16,19,33,0
228587,44,30,18,12,282.11,1015,81,7.2,2020,3,16,19,34,0
228588,85,35,17,18,282.11,1015,81,7.2,2020,3,16,19,34,0


In [29]:
# Get a list of all station numbers as a station_numbers = list(sorted(data.number.unique()))

In [30]:
features= ['number','main_temp', 'main_pressure', 'main_humidity', 'wind_speed','hour','minute',]

In [51]:
#monday data training 
from sklearn import metrics

X = monday[features]
y = monday.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor =RandomForestRegressor(n_estimators=100)

regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('monday_station.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Test Set Predictions for Station: 
        Actual  Predicted
91299        0     0.5100
100076       2     1.4600
84827       24    24.1000
221909      20    19.7075
86106        3     3.0000
...        ...        ...
223501      16    15.1600
97314       23    24.0200
224052      18    17.5400
225108      10    10.0000
99033       14    12.1600

[9319 rows x 2 columns]

Mean Absolute Error: 0.9383582670325346
Mean Squared Error: 2.672393714155838
Root Mean Squared Error: 1.6347457643792316
The accuracy score :  0.9746128835777634


In [32]:
#tuesday data training 
from sklearn import metrics
X = tuesday[features]
y = tuesday.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('tuesday_station.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))



Test Set Predictions for Station: 
        Actual  Predicted
229144      20      19.95
233110      12      11.84
120776      18      18.22
122951      10      11.15
238426       7       7.37
...        ...        ...
113678       1       0.99
118982       2       1.50
231003      25      26.18
112756       1       0.35
230556      13      12.34

[9835 rows x 2 columns]

Mean Absolute Error: 0.9235762764180404
Mean Squared Error: 2.6532337300730906
Root Mean Squared Error: 1.6288749890869743
The accuracy score :  0.9739622943305989


In [33]:
#wednesday data training 
from sklearn import metrics
X = wednesday[features]
y = wednesday.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('wednesday_station.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Test Set Predictions for Station: 
        Actual  Predicted
135014       1       0.97
142047      31      30.16
249746       1       2.22
132455       3       4.43
131510       1       1.27
...        ...        ...
142899       9       9.94
139885       8       7.07
4373        21      20.96
130656       7       7.45
139674       2      12.03

[13854 rows x 2 columns]

Mean Absolute Error: 1.1364677598699362
Mean Squared Error: 3.8795750933843878
Root Mean Squared Error: 1.9696637005804793
The accuracy score :  0.9592580286272256


In [34]:
#thursday data training 
from sklearn import metrics
X = thursday[features]
y = thursday.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('thursday_station.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Test Set Predictions for Station: 
        Actual  Predicted
157438      25      22.30
156014      10       9.24
153365      32      31.95
267718      20      19.90
19288       28      29.91
...        ...        ...
153116       2       1.50
276847      11      11.14
155169      10       9.97
270783      17      16.59
146192      18      18.15

[17334 rows x 2 columns]

Mean Absolute Error: 1.0429664463454704
Mean Squared Error: 3.06807955322604
Root Mean Squared Error: 1.7515934326281428
The accuracy score :  0.96902374045349


In [35]:
#friday data training 
from sklearn import metrics
X = friday[features]
y = friday.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('friday_station.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Test Set Predictions for Station: 
        Actual  Predicted
46430       20      21.80
43492        3       3.95
41847        1       0.92
175938       4       4.53
38380       38      36.96
...        ...        ...
165256      11      11.00
36050        0       1.27
49369        6       7.90
30261       13      12.95
292794      12      10.89

[17157 rows x 2 columns]

Mean Absolute Error: 0.9911289089279122
Mean Squared Error: 2.7995225411615112
Root Mean Squared Error: 1.6731773788697692
The accuracy score :  0.9662522452235793


In [36]:
#saturday data training 
from sklearn import metrics
X = saturday[features]
y = saturday.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('saturday_station.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Test Set Predictions for Station: 
        Actual  Predicted
55032       17      16.97
197778      19      17.61
63806       11       8.34
301149      10       9.98
300080      25      24.68
...        ...        ...
182461       2       2.08
194818       8       4.78
64073        1       1.01
197246      21      20.58
297147      21      21.00

[14116 rows x 2 columns]

Mean Absolute Error: 0.7929466039887193
Mean Squared Error: 2.265968883140077
Root Mean Squared Error: 1.505313549776284
The accuracy score :  0.9690511452498288


In [37]:
#sunday data training 
from sklearn import metrics
X = sunday[features]
y = sunday.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('sunday_station.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Test Set Predictions for Station: 
        Actual  Predicted
83500        3       3.25
214552       3       2.88
209710      22      21.42
67597       14      14.34
67114        9       9.25
...        ...        ...
76614        0       0.98
206547      10      10.57
201492       6       6.28
68983        3       3.37
201691      19      19.23

[10509 rows x 2 columns]

Mean Absolute Error: 0.7345312566552933
Mean Squared Error: 2.050179083097816
Root Mean Squared Error: 1.4318446434923784
The accuracy score :  0.9785576665470587


## Testing pickel files 

In [38]:
row=[0]*7

In [39]:
row

[0, 0, 0, 0, 0, 0, 0]

In [40]:
X = pd.DataFrame([row],columns=features)

In [41]:
inputData = {'number':12,'hour': 10,'minute':30, 'main_temp': 279.60, 'main_pressure': 1025, 'main_humidity': 70, 'wind_speed': 6.6}

In [42]:
X.dtypes

number           int64
main_temp        int64
main_pressure    int64
main_humidity    int64
wind_speed       int64
hour             int64
minute           int64
dtype: object

In [43]:
def dataframe(df, dict):
    df['number'][0]=dict['number']
    df['hour'][0] = dict['hour']
    df['main_temp'][0] = dict['main_temp']
    df['main_pressure'][0] = dict['main_pressure']
    df['main_humidity'][0] = dict['main_humidity']
    df['wind_speed'][0] = dict['wind_speed']
    df['minute'][0] = dict['minute']

In [44]:
dataframe(X,inputData)

In [45]:
X.head()

,number,main_temp,main_pressure,main_humidity,wind_speed,hour,minute
0,12,279,1025,70,6,10,30


In [46]:
# Make a prediction with the input dataframe

with open('tuesday_station.pkl', 'rb') as handle:
    regressor = pickle.load(handle) 
    
prediction = regressor.predict(X)

In [47]:
prediction[0]

13.1